# Advanced Computer Vision - Diego Pinheiro
## Assignment 3 - Camera calibration and 3-D reconstructiion
### Contents

The Equations and Algorithms were adapted from Simon Prince's book and algorithms [1].

## Used packages 

In [1]:
from PIL import Image
from pylab import ginput
import scipy.misc as misc
import scipy.ndimage as ndimage
import scipy.special as sp
import scipy.optimize as op
import scipy.stats as stats
from matplotlib.patches import Rectangle
import numpy as np
import numpy.linalg as la
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = 20, 10
np.set_printoptions(precision=4, suppress=True)

## Part 1: Estimate the camera motion

The solution to this problem is to minimize:

\begin{equation}
\mathbf{\hat{\Omega}}, \mathbf{\hat{\tau}} = \operatorname*{arg\,min}_{\mathbf{\Omega}, \mathbf{\tau}} \Bigg[ \sum_{i=1}^{I} (\mathbf{x}_i - pinhole[w_i, \Lambda, \Omega, \tau])^T (\mathbf{x}_i - pinhole[w_i, \Lambda, \Omega, \tau]) \Bigg]
\end{equation}

In [2]:
def pinhole(wi, camera, rotation, translation):
    return np.dot(camera, np.dot(np.c_[rotation, translation], wi))

In [3]:
def least_squares(X, W, Phi):
    Phi = Phi.reshape(3,4)
    rotation = Phi[:,0:3]
    translation = Phi[:,3:4]
    L = 0
    for i in range(I):
        xi = X[i]
        wi = W[i]
        first_term = (xi - pinhole(wi, camera, rotation, translation)).T
        second_term = (xi - pinhole(wi, camera, rotation, translation))
        L += np.dot(first_term, second_term)
    return L

### Load world and images data points

In [4]:
W = None
with open('datapoints/uvw.txt') as f:
    W = f.readlines()
    W = np.array([np.array(p.split('\t')).reshape(3,1) for p in W],dtype=np.float64)
    
X = None
with open('datapoints/xy.txt') as f:
    X = f.readlines()
    X = np.array([np.array(p.split('\t')).reshape(2,1) for p in X], dtype=np.float64)
    
I = len(W)

In [5]:
W_tilde = np.array([np.vstack([wi, 1]) for wi in W])
X_tilde = np.array([np.vstack([xi, 1]) for xi in X])
X_prime = np.zeros(shape=X_tilde.shape)

### Setting the know camera matrix $\Lambda$

In [6]:
camera = np.array([[200, 0, 256], [0, 200, 256], [0, 0, 1]], dtype=np.float64)
print(camera)

[[ 200.    0.  256.]
 [   0.  200.  256.]
 [   0.    0.    1.]]


### Calculationg the inverse of the camera matrix $\Lambda^{-1}$

In [7]:
camera_inv = np.linalg.inv(camera)
print(camera_inv)

[[ 0.005  0.    -1.28 ]
 [ 0.     0.005 -1.28 ]
 [ 0.     0.     1.   ]]


In [8]:
A = np.zeros((2*I,12))
for i in range(I):
    #Convert to normalized camera coordinates
    wi = W_tilde[i]
    xi = X_tilde[i]
    xip = np.dot(camera_inv, xi)
    X_prime[i] = xip
    A[2*i] = [wi[0], wi[1], wi[2], 1, 0, 0, 0, 0, 
              -wi[0]*xip[0], -wi[1]*xip[0], -wi[2]*xip[0], -xip[0]]
    A[2*i + 1] = [0, 0, 0, 0, wi[0], wi[1], wi[2], 1, 
                  -wi[0]*xip[1], -wi[1]*xip[1], -wi[2]*xip[1], -xip[1]]

### Solve with SVD

In [9]:
U,s,V = np.linalg.svd(A, full_matrices=True)
b = V.T[:,11]

Last column of $\mathbf{V}$ ($\mathbf{b} = \mathbf{v}_{12}$)

In [10]:
print(b)

[-0.004  -0.     -0.0001  0.2623  0.     -0.0041 -0.      0.     -0.001   0.
  0.0004 -0.965 ]


### Extract the first estimate of Rotation Matrix $\Omega$ up to unknown scale using linear method

In [11]:
rotation = np.array([b[0], b[1], b[2], b[4], b[5], b[6], b[8], b[9], b[10]])
rotation = rotation.reshape(3,3)
print(rotation)

[[-0.004  -0.     -0.0001]
 [ 0.     -0.0041 -0.    ]
 [-0.001   0.      0.0004]]


###  Extract the first estimate of Translation Matrix $\tau$ up to unknown scale using linear method

In [12]:
translation = np.array([b[3], b[7], b[11]])
translation = translation.reshape(3,1)
print(translation)

[[ 0.2623]
 [ 0.    ]
 [-0.965 ]]


### Find corrected Rotation Matrix $\hat{\Omega}$ using Procrustes method (still linear solution) 

In [13]:
U,s,V = np.linalg.svd(rotation, full_matrices=True)
rotation_hat = np.dot(U, V)
print(rotation_hat)

[[-0.9685 -0.     -0.2491]
 [ 0.     -1.      0.    ]
 [-0.2491  0.      0.9685]]


### Re-scale the Translation Matrix $\hat{\tau}$ using the estimated scaling factor

In [14]:
scaling_factor = 0
for i in range(3):
    for j in range(3):
        scaling_factor += (rotation_hat[i,j]/rotation[i,j])/9
translation_hat = translation * scaling_factor
print('scaling factor: ' + str(scaling_factor))
print(translation_hat)

scaling factor: 692.835549408
[[ 181.7222]
 [   0.    ]
 [-668.567 ]]


### Check the sing of $\mathbf{\tau}_z$

In [15]:
if translation[2] < 0:
    rotation_hat *= -1
    translation_hat *= -1

### Linear solution for Rotation Matrix $\hat{\Omega}$ 

In [16]:
print(rotation_hat)

[[ 0.9685  0.      0.2491]
 [-0.      1.     -0.    ]
 [ 0.2491 -0.     -0.9685]]


### Linear solution for Translation Matrix $\hat{\tau}$

In [17]:
print(translation_hat)

[[-181.7222]
 [  -0.    ]
 [ 668.567 ]]


### Attempt to reconstruct $x = [175 ~229]^T$ from $w = [-25 ~ -25 ~200]^T$ using the final linear solution 

In [18]:
i = 0
X_tilde_i = np.dot(camera, np.dot(np.hstack([rotation_hat, translation_hat]), W_tilde[i]))
X_i = X_tilde_i / X_tilde_i[2]
print X_i[0:2]

[[ 189.3808]
 [ 245.3309]]


### Perform non-linear optimization using the found linear solution as initial guess

In [19]:
objective_function = (lambda phi: least_squares(X_tilde, W_tilde, phi))
phi_0 = np.hstack([rotation_hat, translation_hat]).flatten()
result = op.minimize(fun=objective_function, x0=phi_0, method='Nelder-Mead')
phi = result.x
phi = phi.reshape(3,4)

### Final Solution for Rotation Matrix $\Omega$ using Non-Linear Optimization

In [20]:
rotation_hat = phi[:,0:3]
print('Rotation Matrix')
print(phi[:,0:3])

Rotation Matrix
[[ 0.0048  0.      0.    ]
 [-0.      0.0046 -0.    ]
 [-0.     -0.     -0.    ]]


### Final Solution for Translation Matrix $\tau$ using Non-Linear Optimization

In [21]:
translation_hat = phi[:,3:4]
print('Translation Matrix')
print(phi[:,3:4])

Translation Matrix
[[-0.2711]
 [-0.    ]
 [ 1.    ]]


### Attempt to reconstruct $x = [175 ~229]^T$ from $w = [-25 ~ -25 ~200]^T$ using the final linear solution 

In [22]:
i = 0
X_tilde_i = np.dot(camera, np.dot(np.hstack([rotation_hat, translation_hat]), W_tilde[i]))
X_i = X_tilde_i / X_tilde_i[2]
print X_i[0:2]

[[ 177.7782]
 [ 232.7771]]


## Part 2: Multi-view triangulation

The solution to this problem is to minimize:

\begin{equation}
\mathbf{\hat{w}} = \operatorname*{arg\,min}_{\mathbf{w}} \Bigg[ \sum_{j=1}^{J} (\mathbf{x}_j - pinhole[w, \Lambda_j, \Omega_j, \tau_j])^T (\mathbf{x}_j - pinhole[w, \Lambda_j, \Omega_j, \tau_j]) \Bigg]
\end{equation}

### Load camera parameters $\{\Lambda_j, \Omega_j, \tau_j\}_{j=1}^{J}$

In [23]:
J = 3
camera = []
rotation = []
translation = []

camera_1 = np.array([[200, 0, 256], 
                     [0, 200, 256], 
                     [0, 0, 1]], dtype=np.float64)
rotation_1 = np.array([[0.81915, 0.00000, 0.57358], 
                       [0, 1, 0], 
                       [-0.57358, 0, 0.81915]],dtype=np.float64)
translation_1 = np.array([[-200, 0, 0]], dtype=np.float64).T
camera.append(camera_1)
rotation.append(rotation_1)
translation.append(translation_1)

camera_2 = np.array([[200, 0, 256], 
                     [0, 200, 256], 
                     [0, 0, 1]], dtype=np.float64)
rotation_2 = np.array([[1, 0, -0], 
                       [0, 1, 0], 
                       [0, 0, 1]], dtype=np.float64)
translation_2 = np.array([[0, 0, 0]], dtype=np.float64).T
camera.append(camera_2)
rotation.append(rotation_2)
translation.append(translation_2)

camera_3 = np.array([[200, 0, 256], 
                     [0, 200, 256], 
                     [0, 0, 1]], dtype=np.float64)
rotation_3 = np.array([[0.81915, 0, -0.57358], 
                       [0, 1, 0], 
                       [0.57358 , 0, 0.81915]], dtype=np.float64)
translation_3 = np.array([[200, 0, 0]], dtype=np.float64).T
camera.append(camera_3)
rotation.append(rotation_3)
translation.append(translation_3)

### Calculationg the inverse of the camera matrix $\Lambda_j^{-1}$

In [24]:
camera_inv = []
for j in range(J):
    camera_inv.append(np.linalg.inv(camera[j]))

### Load image points $\{ x_j \}_{j=1}^J$

In [25]:
X = []
X_tilde = []
for j in range(J):
    X_j = None 
    X_tilde_j = None
    with open('multiview/camera' + str(j+1) + '_pts.txt') as f:
        X_j = f.readlines()
        X_j = [np.array(p.split(' ')) for p in X_j]
        X_j = [p[p != ''] for p in X_j]
        X_j = np.array([p.reshape(2,1) for p in X_j], dtype=np.float64)
        X_tilde_j = np.array([np.vstack([xi, 1]) for xi in X_j])
        X.append(X_j)
        X_tilde.append(X_tilde_j)

I = len(X[0])

### Convert to normalized camera coordinates
\begin{equation}
x_j^{'} = \Lambda_j^{-1} [x_j,y_j,1]^T
\end{equation}

In [26]:
X_prime = []
for j in range(J):
    X_prime_j = [np.dot(camera_inv[j], X_tilde[j][i]) for i in range(I)]
    X_prime.append(X_prime_j)

### Compute linear constraints

In [27]:
W = []
A = []
b = []
for i in range(I):
    A_i = np.zeros((2*J,3))
    b_i = np.zeros((2*J,1))
    for j in range(J):
        w_j = rotation[j]
        t_j = translation[j]
        x_prime_j = X_prime[j][i][0]
        y_prime_j = X_prime[j][i][1]
        A_i[2*j] = [w_j[2,0]*x_prime_j - w_j[0,0], 
                    w_j[2,1]*x_prime_j - w_j[0,1], 
                    w_j[2,2]*x_prime_j - w_j[0,2]]
        A_i[2*j + 1] = [w_j[2,0]*y_prime_j - w_j[1,0], 
                        w_j[2,1]*y_prime_j - w_j[1,1] , 
                        w_j[2,2]*y_prime_j - w_j[1,2]]
        b_i[2*j] = [t_j[0] - t_j[2]*x_prime_j]
        b_i[2*j+ 1] = [t_j[1] - t_j[2]*y_prime_j]
    w_i = np.linalg.lstsq(A_i, b_i)[0]
    W.append(w_i)
    A.append(A_i)
    b.append(b_i)

In [33]:
A[0]

array([[-0.4787, -0.    , -1.0598],
       [ 0.0805, -1.    , -0.1149],
       [-1.    , -0.    , -0.125 ],
       [-0.    , -1.    , -0.125 ],
       [-0.5705,  0.    ,  0.9287],
       [-0.0959, -1.    , -0.137 ]])

### Attempt to reconstruct  $w = [-25 ~ -25 ~200]^T$ from $x = [175 ~229]^T$ using the final linear solution 

In [28]:
W[0]

array([[ -25.    ],
       [ -24.9999],
       [ 199.9995]])

In [29]:
def least_squares(X_i, W_i, camera, rotation, translation):
    L = 0
    wj = np.vstack([W[0],1]).reshape(4,1)
    for j in range(J):
        xj = np.vstack([X_i[j], 1])
        camera_j = camera[j]
        rotation_j = rotation[j]
        translation_j = translation[j]
        first_term = (xj - pinhole(wj, camera_j, rotation_j, translation_j)).T
        second_term = (xj - pinhole(wj, camera_j, rotation_j, translation_j))
        L += np.dot(first_term, second_term)
    return L

In [30]:
W_non_linear = []
for i in range(I):
    X_i = np.array([X[0][i], X[1][i], X[2][i]]) 
    W_i = W[i]
    objective_function = (lambda W_i: least_squares(X_i, W_i, camera, rotation, translation))
    phi_0 = W_i.flatten()
    result = op.minimize(fun=objective_function, x0=phi_0, method='Nelder-Mead')
    phi = result.x
    W_non_linear.append(phi.reshape(3,1))

### Attempt to reconstruct  $w = [-25 ~ -25 ~200]^T$ from $x = [175 ~229]^T$ using the final linear solution 

In [31]:
W_non_linear[0]

array([[ -25.    ],
       [ -24.9999],
       [ 199.9995]])

## References 
[1] Prince, S. J. D. (2012). Computer Vision: Models Learning and Inference. Cambridge University Press.